In [15]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.cloud import storage

#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom-tabular-bq-managed-dataset.ipynb

In [16]:
print(tf.__version__)

2.11.0


In [17]:
#!gsutil cp gs://smart-ml-pongthorn/New_Incident.csv data

# !gsutil -m cp -r "gs://tf1-incident-pongthorn/model" .

In [18]:
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']
isLocalModel=True

localFile=True


# Load and Map Data Test

In [19]:
if  localFile:
    newData_path='../../data/New_Incident.csv'
    dfNewData=pd.read_csv(newData_path)

dfNewData=dfNewData.tail(5)
dfNewData

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,imported_at,open_to_close_hour,response_to_resolved_hour
45,2468,2,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,13.016667,0.866667
46,2451,2,Major,24x7 4Hrs Response Time,Software,Veritas,Incident,Software,2023-03-20 21:00:07.377573,13.000000,12.750000
47,2470,4,Cosmatic,24x7 4Hrs Response Time,Software,Red Hat,Request,General Incident,2023-03-20 21:00:07.377573,102.650000,98.016667
48,2437,4,Cosmatic,24x7 4Hrs Resolution Time,Software,Red Hat,Request,Software,2023-03-20 21:00:07.377573,47.366667,23.016667
49,2279,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,868.283333,868.133333


In [20]:
mapping_file="incident_sevirity_to_class.json"

def download_map_severity_class_as_dict():
 
    storage_client = storage.Client()
    bucket = storage_client.bucket("smart-ml-pongthorn")
    blob = bucket.blob(mapping_file)
    blob.download_to_filename(mapping_file)
    with open(mapping_file, 'r') as file:
            return json.loads(file.read())

if  localFile==False:    
    map_sevirity_to_class= download_map_severity_class_as_dict()
else:
   with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)
                                  
                                  
print(map_sevirity_to_class)


{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [21]:
dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)
print(dfNewData.info())
dfNewData

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 45 to 49
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         5 non-null      int64  
 1   severity_id                5 non-null      int64  
 2   severity                   5 non-null      int64  
 3   severity_name              5 non-null      object 
 4   sla                        5 non-null      object 
 5   product_type               5 non-null      object 
 6   brand                      5 non-null      object 
 7   service_type               5 non-null      object 
 8   incident_type              5 non-null      object 
 9   imported_at                5 non-null      object 
 10  open_to_close_hour         5 non-null      float64
 11  response_to_resolved_hour  5 non-null      float64
dtypes: float64(2), int64(3), object(7)
memory usage: 612.0+ bytes
None


,id,severity_id,severity,severity_name,sla,product_type,brand,service_type,incident_type,imported_at,open_to_close_hour,response_to_resolved_hour
45,2468,2,2,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,13.016667,0.866667
46,2451,2,2,Major,24x7 4Hrs Response Time,Software,Veritas,Incident,Software,2023-03-20 21:00:07.377573,13.000000,12.750000
47,2470,4,0,Cosmatic,24x7 4Hrs Response Time,Software,Red Hat,Request,General Incident,2023-03-20 21:00:07.377573,102.650000,98.016667
48,2437,4,0,Cosmatic,24x7 4Hrs Resolution Time,Software,Red Hat,Request,Software,2023-03-20 21:00:07.377573,47.366667,23.016667
49,2279,3,1,Minor,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,868.283333,868.133333


# Load Model from Directory to Predict

In [79]:
if  isLocalModel: # Window Enviroment
    PATH_FOLDER_ARTIFACTS="model"    
else: # For running on Vertex AI Notebook t
    PATH_FOLDER_ARTIFACTS="gs://tf1-incident-pongthorn/model"
    
model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
print(f"Load from {PATH_FOLDER_ARTIFACTS}")
# model.summary()

Load from model


In [95]:
pdPrediction=pd.DataFrame(columns=['_id','_predict_severity'])
for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}
      # print(input_dict)
    
      predictionResult = model.predict(input_dict)
      print(f"{predictionResult}")   
      # maxResult=np.max(arryResult)
      # maxResult=round( maxResult,4)
      # print(f"{maxResult} of {arryResult}")
        
      prob = tf.nn.softmax(predictionResult)
      prob_pct=(100 * prob)  
      _class = tf.argmax(predictionResult,-1).numpy()[0]
      print(f"{prob_pct} %   as {_class}")   
        
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':_class}])] )
      print("======================================================================================")

dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]
dfPredictData

2468 - 2(Major)
1/1 [==============================] - 0s 33ms/step
[[0.02112172 0.70142525 0.26909646 0.00835664]]
[[19.072248 37.657715 24.439703 18.830338]] %   as 1
2451 - 2(Major)
1/1 [==============================] - 0s 44ms/step
[[0.07360438 0.37683257 0.54041976 0.00914335]]
[[20.463696 27.712414 32.637688 19.186201]] %   as 2
2470 - 0(Cosmatic)
1/1 [==============================] - 0s 39ms/step
[[8.6275071e-01 1.2657920e-01 1.0634896e-02 3.5183642e-05]]
[[42.965107 20.577885 18.325115 18.1319  ]] %   as 0
2437 - 0(Cosmatic)
1/1 [==============================] - 0s 32ms/step
[[9.673976e-02 7.754872e-01 1.271252e-01 6.477430e-04]]
[[20.363949 40.145626 20.992212 18.498213]] %   as 1
2279 - 1(Minor)
1/1 [==============================] - 0s 31ms/step
[[7.99879525e-03 8.69012415e-01 1.22253105e-01 7.35699316e-04]]
[[18.250313 43.17216  20.45928  18.11824 ]] %   as 1


,id,_predict_severity,severity,severity_name
0,2468,1,2,Major
1,2451,2,2,Major
2,2470,0,0,Cosmatic
3,2437,1,0,Cosmatic
4,2279,1,1,Minor


In [25]:
from google.cloud import aiplatform
PROJECT_NUMBER="pongthorn"
ENDPOINT_ID="7022571970512289792"
REGION="asia-southeast1"
endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_NUMBER}/locations/{REGION}/endpoints/{ENDPOINT_ID}")
print(endpoint)

resource name: projects/780371717407/locations/asia-southeast1/endpoints/7022571970512289792


In [29]:
df2=dfNewData.copy().drop(columns=unUsedColtoPredict)
for row in df2.to_dict(orient="records"):
    request_data = {key: [value] for key,value in row.items()}
    # print(request_data)
    response = endpoint.predict([request_data])
    y_predicted = np.argmax(response.predictions, axis=1)
    predictionResult=response[0][0]
    print(predictionResult)
    print(y_predicted)
    
    # print("=======================Prediction response=========================")
    # predictionResult=response[0][0]
    # prob = tf.nn.softmax(predictionResult).numpy()
    # prob_pct=(100 * prob)  
    # max_pct= np.max(prob_pct)
    # index_max_pct=np.argmax(prob, axis = 0)
    # print(f"{prob_pct}: Max={max_pct} as class {index_max_pct}")
    # print("==================================================================")


[0.0211217217, 0.701425254, 0.269096464, 0.00835664291]
[1]
[0.0736043826, 0.376832575, 0.540419757, 0.00914334506]
[2]
[0.862750709, 0.126579195, 0.0106348963, 3.51836425e-05]
[0]
[0.0967397615, 0.775487185, 0.127125204, 0.000647743]
[1]
[0.00799879432, 0.869012296, 0.122253142, 0.000735699956]
[1]
